In [ ]:
# Imports
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neural_network import MLPClassifier
import pandas as pd

In [ ]:
# Set Network Parameters
window_size = 2
dim = 100

In [ ]:
# Retrieve Data
ng = fetch_20newsgroups().data[0:50]

In [ ]:
# Establish Integer indices for each unique term
cv = CountVectorizer()
ng_vecs = cv.fit_transform(ng)
# Store those indices here
vocab = cv.vocabulary_
# And the reverse mapping
id2word = {v:k for (k,v) in vocab.items()}
# The total unique words, aka size of vocabulary
V = len(vocab)

In [ ]:
# Use CountVectorizer to turn our list of documents into a Series of lists of terms
tokenizer = cv.build_tokenizer()
tokenized_docs = pd.Series(ng).map(tokenizer).map(lambda x: [a.lower() for a in x])

In [ ]:
ng[0]

In [ ]:
tokenized_docs[0][0:10]

In [ ]:
X.shape

In [ ]:
# Generate the X data matrix and y vector for MLP
# X: A matrix of one-hot encoded vectors (dimension V) for each center word over all context windows (size 2+2+1=5)
# y: A matrix over all context windows where the outputs are the 4 "labels", aka the indices of the 4 "other" context words
X = []
y = []
# Step thru tokenized document list
for doc in tokenized_docs:
    # For each document, step thru the words
    for index, word in enumerate(doc): 
        # Skip if at the edge of a document (can handle differently)
        if index < 2 or index > (len(doc)-3):
            continue
        # Retrieve the one-hot V-dimensional input vector and add it to X
        one_hot_input = [0]*V 
        one_hot_input[vocab[word]] = 1
        # HACK: Had to do the window cooccurrences separately as MLP won't support multilabel tho it says it does
        X.append(one_hot_input)
        X.append(one_hot_input)
        X.append(one_hot_input)
        X.append(one_hot_input)
        # Retrieve the 4 outputs for the context window and add them to y
        # Same HACK here
        context1 = doc[index-2]
        y.append(vocab[context1])
        context2 = doc[index-1]
        y.append(vocab[context2])
        context3 = doc[index+1]
        y.append(vocab[context3])
        context4 = doc[index+2]
        y.append(vocab[context4])

In [ ]:
# Convert to Numpy arrays
X = np.array(X)
y = np.array(y)

In [ ]:
# Fit the MLP Classifier
mlp = MLPClassifier(hidden_layer_sizes=(dim,))
mlp.fit(X, y)

In [ ]:
# Here are the word vectors!!
word_vecs = mlp.coefs_[0]
word_vecs

In [ ]:
# Look at the closest words to a query using a K-Nearest Neighbors search with cosine
from sklearn.neighbors import NearestNeighbors
nn = NearestNeighbors(metric='cosine', algorithm='brute')
nn.fit(word_vecs)

In [ ]:
# Look at the closest words to a query using a K-Nearest Neighbors search with cosine
def get_similar(query, n=10):
    query_index = vocab[query]
    if query_index:
        dist, index = nn.kneighbors(word_vecs[query_index, :], n_neighbors=n)
        return ([(id2word[i[0]], d[0]) for (d, i) in zip(dist.transpose(), index.transpose())])
    else:
        return "Query not in the dataset!"

In [ ]:
# Try it out!
get_similar("bat", 20)